# 0. Env

In [ ]:
import cv2
import numpy as np
from tqdm.auto import tqdm, trange

from scipy.cluster.vq import kmeans, vq

from datasets import load_dataset, VerificationMode

import matplotlib.pyplot as plt

# 1. Bag of Visual Words
- 참고: https://www.pinecone.io/learn/series/image-search/bag-of-visual-words/

In [ ]:
# 데이터 로딩
data = load_dataset(
    'frgfm/imagenette',
    'full_size',
    split='train',
    verification_mode=VerificationMode.NO_CHECKS
)

In [ ]:
# 데이터 확인
data

In [ ]:
# 첫번째 데이터 확인
data[0]

In [ ]:
# BoW를 학습할 이미지
images_training = []

# 흑백으로 변환해서 확인 (컬러는 너무 많은 리소스를 필요로 함)
# 시간이 너무 오래 걸려서 1000개만 확인
for i in trange(1000):
    img = np.array(data[i]['image'])
    if len(img.shape) == 3:  # BGR 컬러는 흑백으로
        images_training.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    else:
        images_training.append(img)

len(data), len(images_training)

In [ ]:
# 이미지 확인
print(images_training[264].shape)
plt.imshow(images_training[264], cmap='gray')
plt.show()

In [ ]:
# 이미지 확인
print(images_training[874].shape)
plt.imshow(images_training[874], cmap='gray')
plt.show()

In [ ]:
# feature extractor (SIFT)
extractor = cv2.xfeatures2d.SIFT_create()

In [ ]:
# SIFT 특징 추출 (keypoints, descriptors)
keypoints = []
descriptors = []

for img in tqdm(images_training):
    img_keypoints, img_descriptors = extractor.detectAndCompute(img, None)
    keypoints.append(img_keypoints)
    descriptors.append(img_descriptors)

In [ ]:
# 결과 확인
keypoints[0][0].pt, descriptors[0][0]

In [ ]:
# 이미지와 keypoints 출력
for i in range(3):
    output_image = cv2.drawKeypoints(images_training[i],
                                     keypoints[i],
                                     0,
                                     (255, 0, 0),
                                     flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(output_image)
    plt.show()

In [ ]:
# 상황 재현을 위한 random seed 설정
np.random.seed(0)
# 500개 이미지 랜던 선택
sample_idx = np.random.randint(0, len(images_training), 500).tolist()

# 선택된 이미지의 descriptors
descriptors_sample = []

for i in sample_idx:
    descriptors_sample.append(descriptors[i])

In [ ]:
# 선택된 descriptors를 하나의 벡터로 통합
all_descriptors = []
for img_descriptors in descriptors_sample:
    for descriptor in img_descriptors:
        all_descriptors.append(descriptor)
all_descriptors = np.stack(all_descriptors)
all_descriptors.shape

In [ ]:
# cluster to 100 (100개의 keypoints(단어)로 cluster)
k = 100
iters = 1
codebook, variance = kmeans(all_descriptors, k, iters)
codebook.shape, variance

In [ ]:
# 각 이미지의 descriptor를 가장 가까운 codebook으로 할당 (visual word)
visual_words = []
for img_descriptors in descriptors:
    img_visual_words, distance = vq(img_descriptors, codebook)
    visual_words.append(img_visual_words)

In [ ]:
# visual word 확인
visual_words[0][:5], len(visual_words[0])

In [ ]:
# Term Frequeney 계산
term_frequency = []
for img_visual_words in visual_words:
    # Term이 0인 벡터 생성
    img_frequency_vector = np.zeros(k)
    for word in img_visual_words:
        img_frequency_vector[word] += 1  # Term 발생 빈도 증가
    term_frequency.append(img_frequency_vector)
# 하나의 벡터로 통합
frequency_vectors = np.stack(term_frequency)

In [ ]:
# 통합퇸 벡터의 shape
frequency_vectors.shape

In [ ]:
# 첫번째 이미지 TF 값 20개만 확인
frequency_vectors[0][:20]

In [ ]:
# 첫번째 이미지 TF 학인
plt.bar(list(range(k)), frequency_vectors[0])
plt.show()

In [ ]:
# 이미지 숫자 (문서 수)
N = len(frequency_vectors)

# DF 계산
df = np.sum(frequency_vectors > 0, axis=0)

In [ ]:
# 결과 확인
df.shape, df[:5]

In [ ]:
# IDF 계산
idf = np.log(N/ df)
idf.shape, idf[:5]

In [ ]:
# TF-IDF 계산
tfidf = frequency_vectors * idf
tfidf.shape, tfidf[0][:5]

In [ ]:
# 첫번째 이미지 TF-IDF
plt.bar(list(range(k)), tfidf[0])
plt.show()

In [ ]:
# 이미지 검색
top_k = 5
i = 994

# query 이미지 벡터
a = tfidf[i]
# 전체 이미지 벡터
b = tfidf
# 코싸인 유사도 계산
cosine_similarity = np.dot(a, b.T)/(np.linalg.norm(a) * np.linalg.norm(b, axis=1))
# 유사도가 큰 값부터 정렬
idx = np.argsort(-cosine_similarity)[:top_k]
# 결과 출력
for i in idx:
    print(f"{i}: {round(cosine_similarity[i], 4)}")
    plt.imshow(images_training[i], cmap='gray')
    plt.show()